# Quantum Classification

The idea is to work with quantum classification tasks, for the thesis. In this notebook we try to run a very basic version of the quantum classifier. Variational Quantum Classifier (VQC).

## Imports

In [ ]:
from qiskit.version import get_version_info
print(get_version_info())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

In [ ]:
# For EstimatorQNN
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor
from qiskit.algorithms.optimizers import L_BFGS_B


In [ ]:
# For Quantum Convolution Neural Network
import json
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import QuantumCircuit
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import EstimatorQNN, SamplerQNN
from sklearn.model_selection import train_test_split

algorithm_globals.random_seed = 12345

In [ ]:
# For dataset.
from sklearn.datasets import make_multilabel_classification

In [ ]:
# for data split
from sklearn.model_selection import train_test_split
from qiskit_algorithms.utils import algorithm_globals
algorithm_globals.random_seed = 123

In [ ]:
# for classical computing
from sklearn.svm import SVC

In [ ]:
# For Sampling
from qiskit.primitives import Sampler

## Dataset

In [ ]:
# generate fake data
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler

X, y = make_classification(
    n_samples=10000,
    n_features=2,
    n_classes=2,
    n_redundant=0,
    n_clusters_per_class=1,
    class_sep=2.0,
    random_state=algorithm_globals.random_seed,
)

In [ ]:
X.shape, y.shape

In [ ]:
y[:5]

In [ ]:
np.sum(y, axis=0)

In [ ]:
## Train test split
train_features, test_features, train_labels, test_labels = train_test_split(
    X, y, train_size=0.8, random_state=algorithm_globals.random_seed
)

## Building Quantum ML

In [ ]:
num_features = X.shape[1]

In [ ]:
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
feature_map.decompose().draw(output="mpl", fold=20)

In [ ]:
from qiskit.circuit.library import RealAmplitudes

ansatz = RealAmplitudes(num_qubits=num_features, reps=3)
# ansatz.decompose().draw(output="mpl", fold=20)

There are 8 parameters, which are the trainable weights of the classifier.

In [ ]:
# defining the optimiser
optimizer = COBYLA(maxiter=100)

Next we define to train our classifier. We use the instance of Sampler primitive.

In [ ]:
sampler = Sampler()

In [ ]:
objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)


def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

VQC stands for variational quantum classifier. it takes the feature map and an ansatz and constructs a quantum neural network.

In [ ]:
train_labels.shape

In [ ]:
vqc = VQC(
    # sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    loss='cross_entropy',
    optimizer=optimizer,
    callback=callback_graph,
)

# clear objective value history
objective_func_vals = []

start = time.time()
vqc.fit(train_features, train_labels)
elapsed = time.time() - start

print(f"Training time: {round(elapsed)} seconds")

In [ ]:
# testing the QSVC scores
train_score_q4 = vqc.score(train_features, train_labels)
test_score_q4 = vqc.score(test_features, test_labels)

print(f"Quantum VQC on the training dataset: {train_score_q4:.2f}")
print(f"Quantum VQC on the test dataset:     {test_score_q4:.2f}")

To scores are decent, to get better models:
1. The key components are feature map and the ansatz. The `reps` parameter that specifies how many repetitions of a gate pattern we add to the circuit. Larger values lead to more entanglement opearions and more parameters.
2. Changing the optimizers. Gradient based optimizer eg. L_BFGS_B increases the training time. Gradient based optimizer have to evaluate the gradient alongwidth the objective function, which leads to an increase number of circuit execution per iteration.
3. Another option is to sample initial_point and fit the model several times.

## Building SVC first (Classical side)

In [ ]:
svc = SVC()
_ = svc.fit(train_features, train_labels)

In [ ]:
train_score_c4 = svc.score(train_features, train_labels)
test_score_c4 = svc.score(test_features, test_labels)

print(f"Classical SVC on the training dataset: {train_score_c4:.2f}")
print(f"Classical SVC on the test dataset:     {test_score_c4:.2f}")